## Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from google.colab import output#oven timer
import os

##Load Data

In [ ]:
print(os.getcwd())
#Path of the data folder
#data_path = "C:/Users/Admin/OneDrive - McGill University/COMP551550git/NLP-Authorship-Latin-Eng\Data"
data_path = ""
os.path.exists(data_path)

/content


False

In [ ]:

# Load your CSV dataset
training_data = pd.read_csv((data_path + "translated_data_train_16k_format_fix.csv"))
validation_data = pd.read_csv((data_path + "validation_data_filtered.csv"))
test_data = pd.read_csv((data_path + "test_data_filtered.csv"))

train_en_x = training_data['en'].values
train_la_x = training_data['la'].values
valid_en_x = validation_data['en'].values
valid_la_x = validation_data['la'].values
test_en_x  = test_data['en'].values
test_la_x  = test_data['la'].values

train_y = training_data['file'].values
valid_y = validation_data['file'].values
test_y = test_data['file'].values

# Encode labels
label_encoder = OneHotEncoder()
label_encoder.fit(train_y.reshape(-1,1))

train_y=label_encoder.transform(train_y.reshape(-1,1)).toarray()
valid_y =label_encoder.transform(valid_y.reshape(-1,1)).toarray()
test_y= label_encoder.transform(test_y.reshape(-1,1)).toarray()


<class 'numpy.ndarray'>
(363, 5)


numpy.ndarray

## Tokenize

In [ ]:
def tokenize(text_nump, tokenizer):


In [ ]:
# Define parameters
max_len = 300  # maximum sequence length
vocab_size = 128  # ASCII characters

# Tokenize and pad the text sequences
tokenizer_la = tf.keras.preprocessing.text.Tokenizer(char_level=True, filters=None, lower=False)
tokenizer_la.fit_on_texts(train_la_x)

tokenizer_en = tf.keras.preprocessing.text.Tokenizer(char_level=True, filters=None, lower=False)
tokenizer_en.fit_on_texts(train_en_x)

#CONTINUE HERE
train_sequences_la = tokenizer.texts_to_sequences(train_texts)
train_sequences_en =tokenizer.texts_to_sequences(test_texts)
test_sequences =tokenizer.texts_to_sequences(test_texts)
test_sequences_la =tokenizer.texts_to_sequences(test_texts)


train_data = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_len)
test_data = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_len)


In [ ]:

# Build the model
model = models.Sequential()
model.add(layers.Embedding(input_dim=vocab_size, output_dim=16, input_length=max_len))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Accuracy: {test_accuracy}")